# TF-IDF and Linear Regression Demo
https://www.oreilly.com/library/view/feature-engineering-for/9781491953235/ch04.html

Proof of concept for one possible approach, using TF-IDF and ridge regression.

This approach uses no engineered features, and exclusively bases the regression model on the presence or absence of certain words in the respondent's answer to a particular question.  We are not considering other responses, meaning this model is attempting to predict one trait from a single response.  In a final product, we would leverage all of the responses for determining traits.

This notebook is for demonstrating the high-level concepts around converting text into features which can be used for using regression.

In [1]:
import random
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge


In [2]:
# Import the data to a df
train = pd.read_csv('data/siop_ml_train_participant.csv')

# Limit the results to a single answer / score combination for demonstration
train = train.drop(['Respondent_ID', 'open_ended_2', 'open_ended_3', 'open_ended_4', 'open_ended_5', 
         'A_Scale_score', 'O_Scale_score', 'C_Scale_score', 'N_Scale_score'], axis=1)

# Confirm that the data has been imported and is formatted correctly
train.head(3)


,open_ended_1,E_Scale_score
0,"I would change my vacation week, because I am ...",2.250000
1,I would talk to my colleague and see if they w...,4.666667
2,I would feel upset because perhaps I already b...,2.250000


In [3]:
def simple_prep (df, column):
    # Lowercase it all
    df[column].str.lower()
    
    # Remove non-alphanumeric characters
    df[column].replace('[^a-zA-Z0-9]', ' ', regex = True)
    
    return df

prepped_data = simple_prep (train, 'open_ended_1')


In [12]:
def vectorize_training_data (df, column):
    # Set the TF-IDF vectorization settings
    vectorizer = TfidfVectorizer(min_df=5)
    
    # Convert text into vectors
    X = vectorizer.fit_transform(df[column]) 
    
    # return the vectorizer object so we can use it later for evaluation
    return X, vectorizer
    
X, vectorizer = vectorize_training_data (prepped_data, 'open_ended_1')

print(X[0])


  (0, 709)	0.08148980333334543
  (0, 103)	0.06611098222547579
  (0, 390)	0.08877500133530604
  (0, 660)	0.05553590665538873
  (0, 677)	0.07857420551439168
  (0, 70)	0.3604897556594089
  (0, 33)	0.1305087284129816
  (0, 77)	0.16820201970082257
  (0, 198)	0.33640403940164515
  (0, 604)	0.14556477338940635
  (0, 290)	0.13271746418775518
  (0, 427)	0.19711231983512112
  (0, 547)	0.16301025719261872
  (0, 675)	0.11523712986393006
  (0, 94)	0.11749466492809
  (0, 84)	0.13627928011202659
  (0, 69)	0.0658285866062222
  (0, 414)	0.10035942659721815
  (0, 700)	0.198079804788346
  (0, 59)	0.22437438100114485
  (0, 606)	0.15019760255331036
  (0, 529)	0.1371160903087401
  (0, 626)	0.07985488931482768
  (0, 610)	0.10189821294052104
  (0, 430)	0.25094145262797446
  (0, 447)	0.18669882033089774
  (0, 413)	0.1674868586227046
  (0, 433)	0.15365722436421728
  (0, 29)	0.11380033428729315
  (0, 208)	0.18525331498842948
  (0, 83)	0.12774260271556143
  (0, 411)	0.21574941798242342
  (0, 338)	0.21280384721175

In [5]:
# Utilizing ridge regression for this PoC
mod = Ridge(alpha=1.0, random_state=241)

# Set the criterion column so we can fit the model
y = train['E_Scale_score']

# Fit the model using the our training data and criterion column
mod.fit(X, y) 


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

###  Viewing Sample Results

Typically the accuracy of a model would be assessed by utilizing hold-out groups and training on x% of the data, then testing on the remaining portion.  Multiple 'folds' of this evaluation can be performed by randomly assigning the data to be in training or testing set.  This multiple folding process is known as Cross-Validation.

For this model we are simply proofing the idea, and so we are assessing the accuracy of the model by looking at a random sample row and using the model to classify the test row.

In [6]:
def sample_results (row_num, vectorizer):
    print ('User input for row [{0}]: {1}\n'.format(row_num, train['open_ended_1'][row_num]))
    print ('Actual Score: \t\t{0}'.format(train['E_Scale_score'][row_num].round(2)))
    
    # Use the same data transformation process on the sample row provided
    sample_test_data = vectorizer.transform([train['open_ended_1'][row_num]]) 
    
    # Run that transformed vector against the model by using .predict()
    rslt = mod.predict(sample_test_data)
    
    print ('Predicted Score: \t{0}'.format(rslt[0].round(2)))
    print ('\nCriterion StdDev: \t{0}'.format(train['E_Scale_score'].std().round(2)))


When hosted in GitHub, this sample will only show the most recent run, however you can clone this repository to run it locally and sample the results yourself.  The results for this model are not great, but better than guessing.  Standard deviation is provided for comparison.

In [7]:
# To see results for a specific row, change this value to a row index
test_row_index = random.randint(0, len(train['open_ended_1']))

sample_results (test_row_index, vectorizer)


User input for row [514]: I would tell my co-worker if we can't decide between the two of us we need to find another solution. Flipping a coin is fair. Heads, I get the week off. Tails, you get the week off. If the co-worker wasn't agreeing to this, I would just give them the week off and make them look bad for not being fair.

Actual Score: 		4.5
Predicted Score: 	4.03

Criterion StdDev: 	0.79
